# Build HF embeddings

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

# https://huggingface.co/spaces/mteb/leaderboard
sentence_transformer_model = "lier007/xiaobu-embedding-v2" # rank 1 in chinese
# sentence_transformer_model = "Alibaba-NLP/gte-Qwen2-7B-instruct" # rank 2 in chinese

hf_embeddings_model = HuggingFaceEmbeddings(
    model_name=sentence_transformer_model,
    cache_folder="../sentence_transformer_model",
)

# Build HF vector database

In [ ]:
import os
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Qdrant
from langchain_text_splitters import RecursiveCharacterTextSplitter

document_root_path = "../docs"
documents = ["CNS16190-zh_TW.md", "CNS16190-zh_TW_only_provision.md"]
document_idx = 0
embedding_cache_path = "../embedding_cache/"
db_collection_names = ["CNS16190_md_op_emb_1000_200", "CNS16190_md_hf_emb_1000_200"]
db_collection_idx = 1

if os.path.isdir(
    os.path.join(
        embedding_cache_path, "collection", db_collection_names[db_collection_idx]
    )
):
    # database already exists, load it
    hf_vectorstore = Qdrant.from_existing_collection(
        embedding=hf_embeddings_model,
        path=embedding_cache_path,
        collection_name=db_collection_names[db_collection_idx],
    )
else:
    # database does not exist, create it
    loader = UnstructuredMarkdownLoader(
        os.path.join(document_root_path, documents[document_idx]), mode="elements"
    )
    doc = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents=doc)

    hf_vectorstore = Qdrant.from_documents(
        splits,
        embedding=hf_embeddings_model,
        path=embedding_cache_path,
        collection_name=db_collection_names[db_collection_idx],
    )

# Build OpenAI vector database

In [1]:
import os
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Qdrant
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

document_root_path = "../docs"
documents = ["CNS16190-zh_TW.md", "CNS16190-zh_TW_only_provision.md", "ts_103701_split.pdf"]
document_idx = 2
embedding_cache_path = "../embedding_cache/"
db_collection_names = ["CNS16190_md_op_emb_1000_200", "CNS16190_md_hf_emb_1000_200", "TS103701_pdf_op_emb_1000_200"]
db_collection_idx = 2
mode = "pdf"

if mode == "md":
    if os.path.isdir(
        os.path.join(
            embedding_cache_path, "collection", db_collection_names[db_collection_idx]
        )
    ):
        # database already exists, load it
        openai_vectorstore = Qdrant.from_existing_collection(
            embedding=OpenAIEmbeddings(),
            path=embedding_cache_path,
            collection_name=db_collection_names[db_collection_idx],
        )
    else:
        # database does not exist, create it
        loader = UnstructuredMarkdownLoader(
            os.path.join(document_root_path, documents[document_idx]), mode="elements"
        )
        doc = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(doc)

        openai_vectorstore = Qdrant.from_documents(
            splits,
            embedding=OpenAIEmbeddings(),
            path=embedding_cache_path,
            collection_name=db_collection_names[db_collection_idx],
        )

if mode == "pdf":
    if os.path.isdir(
        os.path.join(
            embedding_cache_path, "collection", db_collection_names[db_collection_idx]
        )
    ):
        # database already exists, load it
        openai_vectorstore = Qdrant.from_existing_collection(
            embedding=OpenAIEmbeddings(),
            path=embedding_cache_path,
            collection_name=db_collection_names[db_collection_idx],
        )
    else:
        pdf_loader = PyPDFLoader(os.path.join(document_root_path, documents[document_idx]))
        pdf_doc = pdf_loader.load()
        pdf_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        pdf_splits = pdf_text_splitter.split_documents(documents=pdf_doc)

        openai_vectorstore = Qdrant.from_documents(
            pdf_splits,
            embedding=OpenAIEmbeddings(),
            path=embedding_cache_path,
            collection_name=db_collection_names[db_collection_idx],
        )

In [2]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=openai_vectorstore.as_retriever(), llm=llm
)


In [3]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [11]:
question = """「The purpose of this test case is the conceptual assessment of the manner in which vulnerabilities are acted on a) and the 
confirmation that the preconditions for the implementation are ensured b). 」符合哪一項 Test group？
告訴我他的 Test group objective、Test units 和 Assignment of verdict """

In [12]:
result = retriever_from_llm.invoke(question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 「這個測試案例的目的是評估漏洞的處理方式以及確認實施的前提條件是否得到保障。這符合哪一個測試組別？請提供該測試組的目標、測試單元和判決分配資訊。」', '', '2. 「在這個測試案例中，目的是對漏洞行為進行概念評估和確認實施的前提條件。這屬於哪個測試組？能否說明該測試組的目標、測試單元以及判決的分配？」', '', '3. 「就這個測試案例而言，它的目的是評估漏洞的反應機制和確認實施的必要條件。這適用於哪一測試組？請詳述該測試組的目標、測試單位以及如何分配判決。」']


In [13]:
len(result), result

(12,
 [Document(page_content='acknowledgement of receipt and status updates. \nThe verdict FAIL is assigned otherwise. \n5.2.2 Test group 5.2-2 \n5.2.2.0 Test group objective \nThe test group addresses the provision 5.2-2.  \n5.2.2.1 Test case 5.2-2-1 (conceptual) \nTest purpose \nThe purpose of this test case is the conceptual assessment of the manner in which vulnerabilities are acted on a) and the \nconfirmation that the preconditions fo r the implementation are ensured b). \nTest units \na) The TL shall  assess whether the "Action" and the "Time Frame" of each disclosed vulnerability in \nIXIT 3-VulnTypes facilitate that vulnerabilities  are acted on in a timely manner under consideration of the \nvulnerability disclosure policy according to "Publication of Vulnerability Disclosure Policy" in IXIT 2-UserInfo. \nNOTE 1: The consideration of severity and criticality of the addressed vulnerabilities and the kind of vulnerability \n(e.g. firmware, hardware or software) is helpful.', me

# Build chain

In [3]:
from langchain import hub


prompt = hub.pull(
    "rlm/rag-prompt"
)  # https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=07050030-99d2-5925-9ec6-8a8fb67164d3

In [6]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [7]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever_from_llm | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [13]:
import pandas as pd

df = pd.read_csv("../docs/test_data_from_GoogleSheet.csv")
df

,provision,status,support,detail,test scenario,answer,reason,result
0,5.1-1,MC,NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN
1,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,NaN,NaN
2,5.2-1,M,NaN,製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資...,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,NaN,NaN
3,5.3-13,M,NaN,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,NaN,NaN
4,5.3-16,M,NaN,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,NaN,NaN
5,5.4-1,M,NaN,持久存儲中的敏感安全參數應由設備安全的存儲。,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,NaN,NaN
6,5.4-3,M,NaN,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,NaN,NaN
7,5.4-4,M,NaN,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n...,NaN,NaN
8,5.5-1,M,NaN,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN
9,5.5-5,M,NaN,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN


In [41]:
row = df.iloc[5]
item = f"provision:{row['provision']}，test scenario:{row['test scenario']}，answer:{row['answer']}"
item

'provision:5.4-1，test scenario:安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉後保留資料的任何資料儲存裝置），但製造商也可以對記憶體中的敏感安全參數實施類似的方法。問題：當設備有持久性儲存時：1. 設備如何確保敏感安全參數安全儲存？，answer:所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。'

In [42]:
question = f"Test group {row['provision']} 中，告訴我該 Test group 的 1. objective 和 2. Test units 和 3. Assignment of verdict"

question += "\n\n" + item + "\nresult: 這個test scenario和answer在 Assignment of verdict是PASS或FAIL? \nreason: 告訴我理由\n條列用中文回答以上五個問題"
question


'Test group 5.4-1 中，告訴我該 Test group 的 1. objective 和 2. Test units 和 3. Assignment of verdict\n\nprovision:5.4-1，test scenario:安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉後保留資料的任何資料儲存裝置），但製造商也可以對記憶體中的敏感安全參數實施類似的方法。問題：當設備有持久性儲存時：1. 設備如何確保敏感安全參數安全儲存？，answer:所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。\nresult: 這個test scenario和answer在 Assignment of verdict是PASS或FAIL? \nreason: 告訴我理由\n條列用中文回答以上五個問題'

In [43]:
result = rag_chain.invoke(question)

In [44]:
result.pretty_print()

================================== Ai Message ==================================

1. Test group 5.4-1 的目標是確保安全、可信任的儲存機制能有效保護敏感的安全參數。  
2. 測試單元包括檢查設備如何使用 AES 128 加密演算法來保護機敏資訊的存儲。  
3. 根據所提供的答案，判決為 PASS，因為描述的加密方法符合安全儲存的要求。


# Batch test

In [45]:
queries = []
for row in range(len(df)):
    row = df.iloc[row]
    item = f"provision:{row['provision']}，test scenario:{row['test scenario']}，answer:{row['answer']}"
    question = f"Test group {row['provision']} 中，告訴我該 Test group 的 1. objective 和 2. Test units 和 3. Assignment of verdict"
    question += "\n\n" + item + "\nresult: 這個test scenario和answer在 Assignment of verdict是PASS或FAIL? \nreason: 告訴我理由\n條列用中文回答以上五個問題"
    query = question
    queries.append(query)
queries


['Test group 5.1-1  中，告訴我該 Test group 的 1. objective 和 2. Test units 和 3. Assignment of verdict\n\nprovision:5.1-1 ，test scenario:問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置密碼應符合下列其中一項：1. 每個設備都是唯一的。2. 由使用者定義。請詳細說明密碼是如何建立的。 ，answer:使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。\nresult: 這個test scenario和answer在 Assignment of verdict是PASS或FAIL? \nreason: 告訴我理由\n條列用中文回答以上五個問題',
 'Test group 5.1-3 中，告訴我該 Test group 的 1. objective 和 2. Test units 和 3. Assignment of verdict\n\nprovision:5.1-3，test scenario:與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通常可以在行動應用程式和裝置之間進行。注意：密碼需要始終受到保護，加密是保護密碼的標準方法。 如果密碼未加密，則它們可以簡單地讀取（即明文）並且很容易洩漏。（範例：傳輸過程中的密碼保護通常可以透過 TLS/SSL 隧道連線或 VPN 進行）。問題：1. 當使用者向裝置進行身份驗證時，身份驗證資料在傳輸過程中如何進行加密保護？ ，answer:本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TLS v1.2 或更新的連線保護通道，chiper 套件符合 FIPS 14--3\r\n雲端管理通訊也透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。 設備與雲端之間透過設備證書進行身份驗證。\nresult: 這個test scenario和answer在 Assignment of verdict是PASS或FAIL? \nreason: 告訴我理由\n條列用中文回答以上五個問題',
 'Test

In [46]:
results = rag_chain.batch(queries)

In [47]:
results

[AIMessage(content='1. Objective: 確保所有消費者物聯網裝置的密碼符合唯一性或由使用者定義的要求。  \n2. Test units: 測試單元包括使用者透過向導設定的密碼，且該密碼與MAC地址/SN無相關性。  \n3. Assignment of verdict: 根據描述的控制機制，該測試場景的評判結果應為PASS，因為密碼設定符合唯一性和安全性要求。', response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 1863, 'total_tokens': 1974}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-82fe1c8e-e3c2-4360-b411-181f721073a8-0', usage_metadata={'input_tokens': 1863, 'output_tokens': 111, 'total_tokens': 1974}),
 AIMessage(content='1. Objective: 測試組5.1-3的目的是確保身份驗證資料在使用者和裝置之間傳輸時進行加密保護。  \n2. Test units: 本地驗證使用HMAC-SHA256質詢和HTTPs TLS v1.2或更新的連線進行加密保護，雲端管理通訊也遵循相同標準。  \n3. Assignment of verdict: 該測試的判決為PASS，因為加密傳輸符合保護身份驗證資料的標準要求。', response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 1612, 'total_tokens': 1739}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logpro

In [58]:
filter_results = []
for result in results:
    result = result.content.split("3. ")[1]
    print(result)
    filter_results.append(result)

Assignment of verdict: 根據描述的控制機制，該測試場景的評判結果應為PASS，因為密碼設定符合唯一性和安全性要求。
Assignment of verdict: 該測試的判決為PASS，因為加密傳輸符合保護身份驗證資料的標準要求。
Assignment of verdict: 判定為 PASS，理由是 D-Link 提供了明確的聯絡資訊和漏洞報告狀態更新的流程。
判決分配：PASS，理由是客戶透過產品網頁可獲得定義的支援期資訊，符合要求。
判決分配：根據提供的答案，判決為PASS，因為產品的型號標籤上有相關資訊。
判決分配：PASS，理由是描述的特權控制機制足以促進職責分離、知情需要和特權最小化的原則。
Assignment of verdict: 判定為PASS，因為D-Link產品經過詳細的源碼安全檢查，確認未存在相關的硬編碼安全參數。
Assignment of verdict: 該測試場景的判決為PASS，因為描述的特權控制機制符合職責分離、需要知道和最小化特權的原則。
根據提供的資訊，該測試組的判定結果為PASS，因為描述的特權控制機制足以促進職責分離、需要知道和特權最小化的原則。
判決分配：PASS，因為所描述的特權控制機制足以支持安全性要求。
Assignment of verdict: PASS，因為使用的加密方式符合安全強度模組的要求，並持續檢查以應對潛在漏洞。
判決分配：根據描述的控制機制是否滿足職責分離、需要知曉和最小特權原則，判決為PASS或FAIL。
Assignment of verdict: 判定為PASS，因為所有未使用的介面確實被停用，符合安全控制機制的要求。
判定結果為 PASS，因為描述的權限控制機制符合職責分離、需求知情和最小權限的原則。
判決分配：如果描述的特權控制機制能夠促進職責分離、需要知道和最小特權原則，則判決為通過（PASS）；否則為失敗（FAIL）。
Assignment of verdict: 根據結果，該測試場景的評定為FAIL，理由是設備無外部感測能力，未能滿足透明記錄的要求。
Assignment of verdict: 如果以上功能正常運作，則判定為PASS；否則為FAIL。
判決分配：根據提供的信息，該測試場景的判決應為PASS，因為所有輸入都經過驗證以防止注入。  
根據提供的資

In [59]:
question_data = {
    "provision": df["provision"].tolist(),
    "test scenario": df["test scenario"].tolist(),
    "answer": df["answer"].tolist(),
    "result": filter_results,
}

question_df = pd.DataFrame(question_data)
question_df

,provision,test scenario,answer,result
0,5.1-1,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,Assignment of verdict: 根據描述的控制機制，該測試場景的評判結果應為P...
1,5.1-3,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,Assignment of verdict: 該測試的判決為PASS，因為加密傳輸符合保護身...
2,5.2-1,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,Assignment of verdict: 判定為 PASS，理由是 D-Link 提供了...
3,5.3-13,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,判決分配：PASS，理由是客戶透過產品網頁可獲得定義的支援期資訊，符合要求。
4,5.3-16,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,判決分配：根據提供的答案，判決為PASS，因為產品的型號標籤上有相關資訊。
5,5.4-1,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,判決分配：PASS，理由是描述的特權控制機制足以促進職責分離、知情需要和特權最小化的原則。
6,5.4-3,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,Assignment of verdict: 判定為PASS，因為D-Link產品經過詳細的...
7,5.4-4,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n...,Assignment of verdict: 該測試場景的判決為PASS，因為描述的特權控制...
8,5.5-1,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,根據提供的資訊，該測試組的判定結果為PASS，因為描述的特權控制機制足以促進職責分離、需要知...
9,5.5-5,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,判決分配：PASS，因為所描述的特權控制機制足以支持安全性要求。


In [60]:
question_df.to_csv("A2_generating_only_TS.csv", index=False, encoding="utf-8-sig")